In [1]:
import pandas as pd
df=pd.read_csv('/content/Fake_Real_Data.csv')

In [2]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df.shape

(9900, 2)

In [4]:
df.label.value_counts()

,count
label,
Fake,5000
Real,4900


In [5]:
df['label_num']= df['label'].map({'Fake':0, 'Real':1})
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [6]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [7]:
nlp=spacy.load("en_core_web_lg")

In [10]:
doc=nlp('Top Trump Surrogate BRUTALLY Stabs Him In The')
doc.vector

array([-1.57856748e-01,  1.25356019e-01, -2.78908759e-02,  1.17827654e-01,
        1.48359686e-01, -4.61527221e-02, -3.56287789e-03,  4.73970026e-02,
        1.90093756e-01,  1.97207856e+00, -5.47367483e-02,  5.93899935e-03,
       -6.43442497e-02, -1.38103534e-02, -3.70635018e-02,  2.82203760e-02,
       -5.71669489e-02,  2.61959642e-01,  5.15527464e-02, -6.79962486e-02,
       -6.65142462e-02, -1.27383620e-01, -4.77708764e-02, -7.00937510e-02,
       -9.52804834e-03,  5.74056208e-02, -1.23908482e-01,  1.28829628e-01,
        3.11469920e-02, -1.24872580e-01, -5.25408387e-02,  1.87630743e-01,
       -1.35661259e-01,  1.37369573e-01,  3.47047985e-01, -2.21423626e-01,
       -1.18836261e-01,  2.21961737e-02, -4.86904904e-02, -1.14261121e-01,
       -9.25499946e-03, -5.59728704e-02, -9.72687453e-02, -1.81229368e-01,
        7.32947439e-02, -9.33698118e-02, -7.22729936e-02, -6.80346340e-02,
       -5.40422499e-02, -3.17082517e-02,  2.02256627e-03,  8.24526176e-02,
       -1.49450034e-01, -

In [11]:
doc.vector.shape

(300,)

In [12]:
df['vector']=df['Text'].apply(lambda text: nlp(text).vector)

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2,
    random_state=54
)

In [17]:
import numpy as np
x_train_2d=np.stack(x_train)
x_test_2d=np.stack(x_test)


In [18]:
x_train_2d

array([[-0.11304632,  0.10073841,  0.04373063, ..., -0.06410192,
        -0.08825061,  0.05366249],
       [-0.03691617,  0.11704712, -0.02683996, ..., -0.02874917,
        -0.03646319,  0.06656425],
       [-0.00133934,  0.12742624,  0.05342902, ..., -0.05677974,
        -0.02986474,  0.02971069],
       ...,
       [-0.05736141,  0.09206547, -0.05752354, ..., -0.09781823,
         0.00239365,  0.10524864],
       [-0.04933737,  0.13437341,  0.04348763, ..., -0.08560229,
         0.01690759,  0.03291166],
       [-0.03474531,  0.18337145, -0.07293966, ..., -0.02745017,
        -0.03580733,  0.05297068]], dtype=float32)

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

scaler=MinMaxScaler()
scaled_train_embed=scaler.fit_transform(x_train_2d)
scaled_test_embed=scaler.transform(x_test_2d)

clf=MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [26]:
y_pred=clf.predict(scaled_test_embed)

In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       996
           1       0.96      0.94      0.95       984

    accuracy                           0.95      1980
   macro avg       0.95      0.95      0.95      1980
weighted avg       0.95      0.95      0.95      1980



In [30]:
from sklearn.neighbors import KNeighborsClassifier

clf= KNeighborsClassifier(n_neighbors=5, metric ='euclidean')

clf.fit(x_train_2d, y_train)

KNeighborsClassifier(metric='euclidean')

In [31]:
y_pred=clf.predict(x_test_2d)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       996
           1       0.98      0.99      0.98       984

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980

